## Bagged OMP vs. Base OMP

In [1]:
#!/usr/bin/env python3

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binomtest
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import OrthogonalMatchingPursuit
from joblib import Parallel, delayed
from tqdm import tqdm  # CLI tqdm to avoid Jupyter widget errors
import argparse

# Helper to print tables without pandas
def print_table(headers, rows):
    cols = list(zip(*([headers] + rows)))
    widths = [max(len(str(item)) for item in col) for col in cols]
    fmt = ' | '.join(f'{{:{w}}}' for w in widths)
    sep = '-+-'.join('-' * w for w in widths)
    print(fmt.format(*headers))
    print(sep)
    for row in rows:
        print(fmt.format(*row))

# ----------------------
# Refined Matching Pursuit
# ----------------------
class RefinedMatchingPursuit:
    """
    Orthogonal Matching Pursuit with least-squares re-estimation on support.
    """
    def __init__(self, n_nonzero_coefs=10, tol=None):
        self.n_nonzero_coefs = n_nonzero_coefs
        self.tol = tol
        self.coef_ = None

    def fit(self, X, y):
        omp = OrthogonalMatchingPursuit(
            n_nonzero_coefs=self.n_nonzero_coefs,
            tol=self.tol
        )
        omp.fit(X, y)
        support = np.flatnonzero(omp.coef_)
        coef_full = np.zeros(X.shape[1])
        if support.size > 0:
            X_sel = X[:, support]
            beta_sel, *_ = np.linalg.lstsq(X_sel, y, rcond=None)
            coef_full[support] = beta_sel
        self.coef_ = coef_full
        return self

    def predict(self, X):
        return X.dot(self.coef_)

# -----------------------------
# Bagged Refined MP with OOB weighting
# -----------------------------
class BaggedRefinedMP:
    """
    Bagged ensemble of RefinedMatchingPursuit:
      - Bootstrap rows only
      - Coefficients aggregated by inverse OOB-error weighting
    """
    def __init__(self, n_estimators=20, n_nonzero_coefs=10,
                 bootstrap=True, random_state=42, n_jobs=-1):
        self.n_estimators = n_estimators
        self.n_nonzero_coefs = n_nonzero_coefs
        self.bootstrap = bootstrap
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.estimators_ = []
        self.oob_errors_ = []

    def _fit_one(self, X, y, seed):
        rng = np.random.RandomState(seed)
        n = len(X)
        if self.bootstrap:
            idx = rng.choice(n, n, replace=True)
            oob = np.setdiff1d(np.arange(n), idx)
            Xb, yb = X[idx], y[idx]
        else:
            oob = np.array([], dtype=int)
            Xb, yb = X, y
        mp = RefinedMatchingPursuit(n_nonzero_coefs=self.n_nonzero_coefs)
        mp.fit(Xb, yb)
        if oob.size > 0:
            y_oob = mp.predict(X[oob])
            err = mean_squared_error(y[oob], y_oob)
        else:
            err = np.nan
        return mp, err

    def fit(self, X, y):
        seeds = [self.random_state + i for i in range(self.n_estimators)]
        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self._fit_one)(X, y, s) for s in seeds
        )
        self.estimators_, self.oob_errors_ = zip(*results)
        return self

    def predict(self, X):
        preds = np.stack([mp.predict(X) for mp in self.estimators_], axis=0)
        return preds.mean(axis=0)

    def bagged_coef(self, n_features):
        coefs = np.stack([mp.coef_ for mp in self.estimators_], axis=0)
        errs = np.array(self.oob_errors_)
        weights = 1.0 / (errs + 1e-8)
        weights /= weights.sum()
        return np.average(coefs, axis=0, weights=weights)

# -----------------------------
# Cross-Validation Utilities
# -----------------------------
def cv_validation_curve(X, y, param_name, param_range, model_ctor, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=0)
    means, stds = [], []
    for v in param_range:
        mses = []
        for tr, te in kf.split(X):
            model = model_ctor(**{param_name: v})
            model.fit(X[tr], y[tr])
            mses.append(mean_squared_error(y[te], model.predict(X[te])))
        means.append(np.mean(mses))
        stds.append(np.std(mses))
    return means, stds


def cv_coef_error_curve(X, y, param_name, param_range, model_ctor, coef_true, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=0)
    means, stds = [], []
    for v in param_range:
        errs = []
        for tr, _ in kf.split(X):
            model = model_ctor(**{param_name: v})
            model.fit(X[tr], y[tr])
            # coefficient vector
            if hasattr(model, 'coef_'):
                est = model.coef_
            else:
                est = model.bagged_coef(X.shape[1])
            errs.append(np.linalg.norm(coef_true - est))
        means.append(np.mean(errs))
        stds.append(np.std(errs))
    return means, stds

# -----------------------------
# Main: Cross-Validated MSE & Coef Error Comparison
# -----------------------------
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--estimators', type=int, default=20, help='Number of bagging estimators')
    parser.add_argument('--folds', type=int, default=5, help='Number of CV folds')
    parser.add_argument('--sparsity', nargs='+', type=int, default=[5,10,15,20,30], help='List of sparsity levels')
    args, _ = parser.parse_known_args()

    # Generate data and true coefficients
    X, y, coef_true = make_regression(
        n_samples=200,
        n_features=100,
        n_informative=15,
        noise=0.5,
        coef=True,
        random_state=0
    )

    # 1) CV-MSE
    omp_means, omp_stds = cv_validation_curve(
        X, y, 'n_nonzero_coefs', args.sparsity,
        lambda n_nonzero_coefs: RefinedMatchingPursuit(n_nonzero_coefs),
        k=args.folds
    )
    bag_means, bag_stds = cv_validation_curve(
        X, y, 'n_nonzero_coefs', args.sparsity,
        lambda n_nonzero_coefs: BaggedRefinedMP(
            n_estimators=args.estimators,
            n_nonzero_coefs=n_nonzero_coefs
        ),
        k=args.folds
    )
    # 2) CV Coefficient Error
    omp_coef_means, omp_coef_stds = cv_coef_error_curve(
        X, y, 'n_nonzero_coefs', args.sparsity,
        lambda n_nonzero_coefs: RefinedMatchingPursuit(n_nonzero_coefs),
        coef_true, k=args.folds
    )
    bag_coef_means, bag_coef_stds = cv_coef_error_curve(
        X, y, 'n_nonzero_coefs', args.sparsity,
        lambda n_nonzero_coefs: BaggedRefinedMP(
            n_estimators=args.estimators,
            n_nonzero_coefs=n_nonzero_coefs
        ),
        coef_true, k=args.folds
    )

    # Print CV-MSE table
    headers_mse = ['Sparsity','OMP CV-MSE','Bagged CV-MSE']
    rows_mse = [[
        s,
        f"{round(omp_means[i],2)}±{round(omp_stds[i],2)}",
        f"{round(bag_means[i],2)}±{round(bag_stds[i],2)}"
    ] for i, s in enumerate(args.sparsity)]
    print_table(headers_mse, rows_mse)

    # Print CV Coefficient Error table
    headers_coef = ['Sparsity','OMP CoefErr','Bagged CoefErr']
    rows_coef = [[
        s,
        f"{round(omp_coef_means[i],2)}±{round(omp_coef_stds[i],2)}",
        f"{round(bag_coef_means[i],2)}±{round(bag_coef_stds[i],2)}"
    ] for i, s in enumerate(args.sparsity)]
    print("\nCoefficient Error Comparison:")
    print_table(headers_coef, rows_coef)

    print("\nCross-validated comparison complete.")


Sparsity | OMP CV-MSE      | Bagged CV-MSE  
---------+-----------------+----------------
       5 | 23622.6±7009.37 | 14962.2±2114.11
      10 | 3227.92±320.78  | 1721.43±231.6  
      15 | 0.32±0.07       | 0.37±0.13      
      20 | 0.36±0.06       | 0.31±0.06      
      30 | 0.39±0.06       | 0.34±0.05      

Coefficient Error Comparison:
Sparsity | OMP CoefErr | Bagged CoefErr
---------+-------------+---------------
       5 | 136.4±9.99  | 115.53±5.64   
      10 | 54.12±1.83  | 41.92±1.86    
      15 | 0.17±0.03   | 0.18±0.03     
      20 | 0.31±0.01   | 0.24±0.02     
      30 | 0.42±0.02   | 0.34±0.01     

Cross-validated comparison complete.
